In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.preprocessing import FunctionTransformer

#ML libraries
import xgboost
from xgboost import XGBRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

#Basic Libraries to have
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv as csv

df = pd.read_csv('df_2102.csv')

sensor_id = [329, 332, 335]
freezer_id = [344, 362, 368]

,gw_time,gw_time.1,status_329,avg_temperature_329,avg_humidity_329,temperature_329,humidity_329,battery_329,available_329,status_332,...,run_time_368,dT_329,dT_332,dT_335,temperature_329_prime,dT_329_prime,temperature_332_prime,dT_332_prime,temperature_335_prime,dT_335_prime
0,2021-02-01 00:00:04,2021-02-01 00:00:04,1,-20.27,59.98,-20.30,63.48,100,1,1,...,0.0,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-02-01 00:00:34,2021-02-01 00:00:34,1,-20.28,59.94,-20.31,63.46,100,1,1,...,1.0,-0.01,-0.01,0.01,-20.30,0.00,-20.23,0.00,-20.28,0.00
2,2021-02-01 00:01:04,2021-02-01 00:01:04,1,-20.28,59.98,-20.31,63.42,100,1,1,...,1.0,0.00,-0.01,0.00,-20.31,-0.01,-20.24,-0.01,-20.27,0.01
3,2021-02-01 00:01:35,2021-02-01 00:01:35,1,-20.27,60.02,-20.31,63.39,100,1,1,...,2.0,0.00,0.02,0.00,-20.31,0.00,-20.25,-0.01,-20.27,0.00
4,2021-02-01 00:02:05,2021-02-01 00:02:05,1,-20.27,60.05,-20.30,63.36,100,1,1,...,2.0,0.01,0.00,0.00,-20.31,0.00,-20.23,0.02,-20.27,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80124,2021-02-28 23:57:33,2021-02-28 23:57:33,1,-19.81,46.37,-18.94,45.02,100,1,1,...,1437.0,0.00,0.01,0.01,-18.94,0.01,-19.82,0.00,-20.68,0.00
80125,2021-02-28 23:58:03,2021-02-28 23:58:03,1,-19.81,46.37,-18.95,44.98,100,1,1,...,1438.0,-0.01,0.01,0.00,-18.94,0.00,-19.81,0.01,-20.67,0.01
80126,2021-02-28 23:58:33,2021-02-28 23:58:33,1,-19.79,46.37,-18.93,45.00,100,1,1,...,1438.5,0.02,0.00,0.02,-18.95,-0.01,-19.80,0.01,-20.67,0.00
80127,2021-02-28 23:59:03,2021-02-28 23:59:03,1,-19.79,46.35,-18.95,44.98,100,1,1,...,1439.0,-0.02,0.01,0.01,-18.93,0.02,-19.80,0.00,-20.65,0.02


In [6]:
df.columns

Index(['gw_time', 'gw_time.1', 'status_329', 'avg_temperature_329',
       'avg_humidity_329', 'temperature_329', 'humidity_329', 'battery_329',
       'available_329', 'status_332', 'avg_temperature_332',
       'avg_humidity_332', 'temperature_332', 'humidity_332', 'battery_332',
       'available_332', 'status_335', 'avg_temperature_335',
       'avg_humidity_335', 'temperature_335', 'humidity_335', 'battery_335',
       'available_335', 'status_344', 'fan_run_344', 'fan_trip_344',
       'freez_run_344', 'freez_trip_344', 'defrost_344', 'ctrl_mode_344',
       'ctrl_operation_344', 'operation_344', 'run_time_344', 'status_362',
       'fan_run_362', 'fan_trip_362', 'freez_run_362', 'freez_trip_362',
       'defrost_362', 'ctrl_mode_362', 'ctrl_operation_362', 'operation_362',
       'run_time_362', 'status_368', 'fan_run_368', 'fan_trip_368',
       'freez_run_368', 'freez_trip_368', 'defrost_368', 'ctrl_mode_368',
       'ctrl_operation_368', 'operation_368', 'run_time_368', 'dT_3

In [59]:
cols = ['gw_time'] + ['operation_'+str(_id) for _id in freezer_id] + ['temperature_335', 'temperature_335_prime']

In [60]:
df = df[cols]

In [61]:
df['gw_time'] = pd.to_datetime(df['gw_time'])

C:\Users\Intellicode\AppData\Local\Temp\ipykernel_4692\2673046131.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gw_time'] = pd.to_datetime(df['gw_time'])


In [62]:
df = df.dropna()
df.isnull().sum()

gw_time                  0
operation_344            0
operation_362            0
operation_368            0
temperature_335          0
temperature_335_prime    0
dtype: int64

In [63]:
df.corr()

C:\Users\Intellicode\AppData\Local\Temp\ipykernel_4692\1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,operation_344,operation_362,operation_368,temperature_335,temperature_335_prime
operation_344,1.000000,0.438878,0.247876,0.325282,0.326585
operation_362,0.438878,1.000000,0.535200,0.351468,0.355176
operation_368,0.247876,0.535200,1.000000,0.463526,0.466595
temperature_335,0.325282,0.351468,0.463526,1.000000,0.999701
temperature_335_prime,0.326585,0.355176,0.466595,0.999701,1.000000


In [67]:
feature = ['operation_344','operation_362','operation_368','temperature_335_prime']
label = ['temperature_335']

In [68]:
X = df[feature]
y = df[label]

In [69]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))
train_X = scaler.fit_transform(X)

In [71]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y= train_test_split(train_X,y,test_size=0.15, random_state=0)

In [74]:
from xgboost import XGBRegressor

my_model = XGBRegressor(n_estimators = 500, max_depth = 9,min_child_weight= 1.2,colsample_bytree = 1.0)
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(train_X, train_y, verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=9, max_leaves=None,
             min_child_weight=1.2, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [ ]:
xgb.d